# Cambiamenti

- Provare ad usare un solo hidden layer
- Provare a cambiare funzione d'attivazione dell'output layer da lineare a ReLu
- Cambiare l'ultimo grafico


In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time
import pickle

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD, RMSprop
from keras.regularizers import l1, l2

from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
#                         project_name="general", workspace="paologalligit")

In [22]:
df = pd.read_csv('fifa19.csv')

In [23]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [24]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)

Individuare eventuali outlier nella colonna value

In [25]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [26]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [27]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [28]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [29]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [30]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
41,88,88,4000000.0,77000.0,NaN,NaN,NaN,79.0,7400000.0
102,85,85,9000000.0,38000.0,70.0,70.0,70.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,63.0,63.0,63.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,63.0,63.0,63.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,NaN,NaN,NaN,78.0,24700000.0


Mescolo le righe del dataset

In [31]:
df_flt = df_flt.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [32]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [33]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [34]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

scaler_test = StandardScaler().fit(y_test_full)
y_test_scaled = scaler_test.transform(y_test_full)

In [35]:
with open('X_scaled', 'wb') as X_scaled_file:
    pickle.dump(X_scaled, X_scaled_file)
    
with open('y_scaled', 'wb') as y_scaled_file:
    pickle.dump(y_scaled, y_scaled_file)

In [36]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

### Solita rete con iperparametri scelti in base a prove empiriche

In [37]:
def build_nn(neurons):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.1))

    # Adding the first hidden layer
    model.add(Dense(128, activation = 'relu'))#, kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.1))

    # Adding the second hidden layer
    model.add(Dense(64, activation = 'relu'))#, kernel_regularizer=l2(0.001)))

    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))

    return model

### Rete scelta con iperparametri ottimali

In [38]:
def data():
    with open('X_scaled', 'rb') as X_scaled_file:
        X_scaled_records = pickle.load(X_scaled_file)

    with open('y_scaled', 'rb') as y_scaled_file:
        y_scaled_records = pickle.load(y_scaled_file)
        
    X_train, X_val, y_train, y_val = train_test_split(X_scaled_records, y_scaled_records, test_size = 0.2, random_state = 42)
    
    return X_train, y_train, X_val, y_val

# data()[0].shape

In [44]:
def create_model(X_train, y_train, X_val, y_val):
    def coeff_determination(y_test, y_pred):
        from keras import backend as K
        SS_res =  K.sum(K.square( y_test-y_pred ))
        SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
        return ( 1 - SS_res/(SS_tot + K.epsilon()))
    
    model = Sequential()
    
    model.add(Dense({{choice([128, 256, 512, 1024])}}, activation='relu', input_shape=(X_train.shape[1],)))
    
    model.add(Dense({{choice([128, 256, 512, 1024])}}, activation='relu'))
    
    model.add(Dense(1, activation='relu'))
    
    opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    model.compile(loss='mean_absolute_error', metrics=[coeff_determination], optimizer=opt)

    model.fit(X_train, y_train,
              batch_size=125,
              epochs=50,
              verbose=1,
              validation_data=(X_val, y_val))
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    
    return {'loss': acc, 'status': STATUS_OK, 'score': score, 'model': model}

In [45]:
%%time

best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='ProvaRegressione_v4_model_selection')

print("Evalutation of best performing model:")
print(best_model.evaluate(X_test_scaled, y_test_scaled))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from re import sub
except:
    pass

try:
    from time import time
except:
    pass

try:
    import pickle
except:
    pass

try:
    from comet_ml import Experiment
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, BatchNormalization, Dropout, Activation
except:
    pass

try:
    from keras.utils import normalize
except:
    pass

try:
    from keras.wrappers.scikit_learn import KerasRegressor
except:
    pass

try:
    from keras.callbacks import TensorBoard
except:
    pass

try:
    from keras.optimizers import Adam, SGD, RMSprop
except:
    pass

try:
    from keras.regularizers import l1, l2
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STAT

 - ETA: 0s - loss: 0.6733 - coeff_determination: -0.0129
                                                  
 5875/11142 [==============>...............]       
 - ETA: 0s - loss: 0.6769 - coeff_determination: -0.0118
                                                  
 6500/11142 [================>.............]       
 - ETA: 0s - loss: 0.6795 - coeff_determination: -0.0114
                                                  
 7125/11142 [==================>...........]       
 - ETA: 0s - loss: 0.6778 - coeff_determination: -0.0117
                                                  
 7625/11142 [===================>..........]       
 - ETA: 0s - loss: 0.6795 - coeff_determination: -0.0112
                                                  
 8375/11142 [=====================>........]       
 - ETA: 0s - loss: 0.6805 - coeff_determination: -0.0107
                                                  
 9125/11142 [=======================>......]       
 - ETA: 0s - loss: 0.6799 - coeff_determ

                                                  
11000/11142 [============================>.]       
 - ETA: 0s - loss: 0.6859 - coeff_determination: -0.0094
                                                  
11142/11142 [==============================]       
 - 1s 112us/step - loss: 0.6861 - coeff_determination: -0.0118 - val_loss: 0.6835 - val_coeff_determination: -0.0042

Epoch 5/50                                         
  125/11142 [..............................]       
 - ETA: 0s - loss: 0.7518 - coeff_determination: -0.0174
                                                  
  875/11142 [=>............................]       
 - ETA: 0s - loss: 0.6615 - coeff_determination: -0.0151
                                                  
 1250/11142 [==>...........................]       
 - ETA: 0s - loss: 0.6528 - coeff_determination: -0.0176
                                                  
 1875/11142 [====>.........................]       
 - ETA: 0s - loss: 0.6616 - coeff_d

                                                  
 6625/11142 [================>.............]       
 - ETA: 0s - loss: 0.6918 - coeff_determination: -0.0098
                                                  
 7250/11142 [==================>...........]       
 - ETA: 0s - loss: 0.6883 - coeff_determination: -0.0093
                                                  
 7875/11142 [====================>.........]       
 - ETA: 0s - loss: 0.6878 - coeff_determination: -0.0093
                                                  
 8500/11142 [=====================>........]       
 - ETA: 0s - loss: 0.6880 - coeff_determination: -0.0091
                                                  
 9250/11142 [=======================>......]       
 - ETA: 0s - loss: 0.6857 - coeff_determination: -0.0093
                                                  
 9875/11142 [=========================>....]       
 - ETA: 0s - loss: 0.6837 - coeff_determination: -0.0092
                                        

                                                  
 2625/11142 [======>.......................]       
 - ETA: 0s - loss: 0.6662 - coeff_determination: -0.0047
                                                  
 3250/11142 [=======>......................]       
 - ETA: 0s - loss: 0.6674 - coeff_determination: -0.0039
                                                  
 3875/11142 [=========>....................]       
 - ETA: 0s - loss: 0.6793 - coeff_determination: -0.0046
                                                  
 4625/11142 [===========>..................]       
 - ETA: 0s - loss: 0.6852 - coeff_determination: -0.0046
                                                  
 5250/11142 [=============>................]       
 - ETA: 0s - loss: 0.6855 - coeff_determination: -0.0050
                                                  
 5750/11142 [==============>...............]       
 - ETA: 0s - loss: 0.6882 - coeff_determination: -0.0056
                                        

...

                                                                              
10625/11142 [===========================>..]                                   
 - ETA: 0s - loss: 0.6889 - coeff_determination: -0.0089                       
                                                                              
11142/11142 [==============================]                                   
 - 1s 83us/step - loss: 0.6861 - coeff_determination: -0.0088 - val_loss: 0.6835 - val_coeff_determination: -0.0042

Epoch 40/50                                                                    
  125/11142 [..............................]                                   
 - ETA: 0s - loss: 0.7474 - coeff_determination: -0.0053                       
                                                                              
  875/11142 [=>............................]                                   
 - ETA: 0s - loss: 0.6622 - coeff_determination: -0.0129                       
      

 - ETA: 0s - loss: 0.6862 - coeff_determination: -0.0110                       
                                                                              
10500/11142 [===========================>..]                                   
 - ETA: 0s - loss: 0.6874 - coeff_determination: -0.0108                       
                                                                              
11125/11142 [============================>.]                                   
 - ETA: 0s - loss: 0.6861 - coeff_determination: -0.0104                       
                                                                              
11142/11142 [==============================]                                   
 - 1s 85us/step - loss: 0.6861 - coeff_determination: -0.0104 - val_loss: 0.6835 - val_coeff_determination: -0.0042

Epoch 42/50                                                                    
  125/11142 [..............................]                                   
 - ETA

 9500/11142 [========================>.....]                                   
 - ETA: 0s - loss: 0.6832 - coeff_determination: -0.0093                       
                                                                              
10250/11142 [==========================>...]                                   
 - ETA: 0s - loss: 0.6871 - coeff_determination: -0.0090                       
                                                                              
11125/11142 [============================>.]                                   
 - ETA: 0s - loss: 0.6859 - coeff_determination: -0.0099                       
                                                                              
11142/11142 [==============================]                                   
 - 1s 78us/step - loss: 0.6861 - coeff_determination: -0.0099 - val_loss: 0.6835 - val_coeff_determination: -0.0042

Epoch 44/50                                                                    
  125/

                                                                              
 8125/11142 [====================>.........]                                   
 - ETA: 0s - loss: 0.6796 - coeff_determination: -0.0113                       
                                                                              
 9250/11142 [=======================>......]                                   
 - ETA: 0s - loss: 0.6812 - coeff_determination: -0.0110                       
                                                                              
10375/11142 [==========================>...]                                   
 - ETA: 0s - loss: 0.6852 - coeff_determination: -0.0101                       
                                                                              
11142/11142 [==============================]                                   
 - 1s 50us/step - loss: 0.6861 - coeff_determination: -0.0098 - val_loss: 0.6835 - val_coeff_determination: -0.0042

Epoch 4

 - ETA: 0s - loss: 0.6975 - coeff_determination: -0.0080                       
                                                                              
 7000/11142 [=================>............]                                   
 - ETA: 0s - loss: 0.6934 - coeff_determination: -0.0079                       
                                                                              
 7625/11142 [===================>..........]                                   
 - ETA: 0s - loss: 0.6917 - coeff_determination: -0.0086                       
                                                                              
 8500/11142 [=====================>........]                                   
 - ETA: 0s - loss: 0.6890 - coeff_determination: -0.0080                       
                                                                              
 9000/11142 [=======================>......]                                   
 - ETA: 0s - loss: 0.6882 - coeff_determinat

In [46]:
best_model, best_run

(<keras.engine.sequential.Sequential at 0x7f50c811c2b0>,
 {'Dense': 1, 'Dense_1': 2})

In [22]:
def build_nn(neurons):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='selu'))#, kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.9662681038993752))

    # Adding the first hidden layer
    model.add(Dense(1024, activation = 'selu'))#, kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.011106434718081704))

    # Adding the second hidden layer
    model.add(Dense(128, activation = 'selu'))#, kernel_regularizer=l2(0.001)))

    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))

    return model

In [23]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [24]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [25]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
kfold = KFold(n_splits=2, random_state=42, shuffle=False)

In [26]:
scores = []
abs_errors = []

nn = build_nn(X_scaled.shape[1])

opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# opt = SGD(lr=0.01, momentum=0.9)
nn.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

In [29]:
%%time

stage = 0

for train, test in kfold.split(X_scaled):
    
    stage += 1
    
    print('#'*60)
    print('\nSTAGE --> {}\n'.format(stage))
    print('#'*60)
    
    X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]
        
    nn.fit(X_train_round, y_train_round, batch_size=512, epochs=100, shuffle=True)
    
    # predict
    res = nn.predict(X_test_round)
    print('R2 of round: ', round(r2_score(y_test_round, res), 4))
    
    # evaluate
    loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)
    
    reversed_res = scaler_train.inverse_transform(res)
    reversed_label = scaler_train.inverse_transform(y_test_round)
    abs_errors.append(mean_abs_error(reversed_res, reversed_label))
    
    print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
    if acc < 0:
        print('X_test round:\n')
        print(X_test_round)
        print('#'*60)
        print('y_test_round:\n')
        print(y_test_round)
        print('#'*60)
    
    scores.append(acc * 100)

############################################################

STAGE --> 1

############################################################
Epoch 1/100
6964/6964 [==============================] - 1s 84us/step - loss: 0.6149 - coeff_determination: -6.5724
Epoch 2/100
6964/6964 [==============================] - 1s 85us/step - loss: 0.5717 - coeff_determination: -2.2526
Epoch 3/100
6964/6964 [==============================] - 1s 84us/step - loss: 0.5999 - coeff_determination: -4.7058
Epoch 4/100
6964/6964 [==============================] - 1s 82us/step - loss: 0.5944 - coeff_determination: -4.9970
Epoch 5/100
6964/6964 [==============================] - 1s 80us/step - loss: 0.5662 - coeff_determination: -0.7569
Epoch 6/100
6964/6964 [==============================] - 1s 84us/step - loss: 0.5522 - coeff_determination: -0.3292
Epoch 7/100
6964/6964 [==============================] - 1s 88us/step - loss: 0.5945 - coeff_determination: -4.5249
Epoch 8/100
6964/6964 [=============================

6964/6964 [==============================] - 1s 92us/step - loss: 0.5487 - coeff_determination: -0.2403
Epoch 70/100
6964/6964 [==============================] - 1s 86us/step - loss: 0.5516 - coeff_determination: -0.2497
Epoch 71/100
6964/6964 [==============================] - 1s 84us/step - loss: 0.5545 - coeff_determination: -0.3839
Epoch 72/100
6964/6964 [==============================] - 1s 87us/step - loss: 0.5614 - coeff_determination: -0.1736
Epoch 73/100
6964/6964 [==============================] - 1s 88us/step - loss: 0.5544 - coeff_determination: -0.1671
Epoch 74/100
6964/6964 [==============================] - 1s 89us/step - loss: 0.5488 - coeff_determination: -0.1697
Epoch 75/100
6964/6964 [==============================] - 1s 85us/step - loss: 0.5519 - coeff_determination: -0.1635
Epoch 76/100
6964/6964 [==============================] - 1s 79us/step - loss: 0.5552 - coeff_determination: -0.5170
Epoch 77/100
6964/6964 [==============================] - 1s 81us/step - loss

6964/6964 [==============================] - 1s 73us/step
6964 6964 6964
loss: 0.5617, acc: -21.8525%
X_test round:

[[ 0.98290807  0.54644819  1.03187083 -0.02796988  0.68786212]
 [ 0.51361366  0.89714191 -0.12124447  0.08973916 -0.12057862]
 [ 2.54722277  1.94922306  8.03926381  2.20850177  3.98717432]
 ...
 [ 2.23435983  1.59852935  1.65277907  1.50224757  2.0862461 ]
 [-0.73783809 -1.73306097 -0.47604918 -0.26338794 -0.61219799]
 [-0.42497515 -1.38236726 -0.47604918  1.03141143 -0.56434704]]
############################################################
y_test_round:

[[ 0.57136149]
 [-0.07517189]
 [ 4.36974514]
 ...
 [ 1.74320326]
 [-0.60048027]
 [-0.52976568]]
############################################################
CPU times: user 5min 21s, sys: 8.15 s, total: 5min 29s
Wall time: 1min 56s


In [30]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores) * 100))
scores, np.mean(abs_errors)

-14.38% (+/- 747.12%)


([-6.9101491719398735, -21.852504857855923], 1415059.075)

In [31]:
res = nn.predict(X_test_scaled)
res = scaler_train.inverse_transform(res)
mean_abs_error(res, y_test.values), max(y_test.values), round(np.mean(y_test.values), 2), round(r2_score(y_test.values, res), 4)

3483 3483 3483


(1365410.94, array([13500000.]), 1691933.68, -0.1925)